# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

filepath = "../output_data/cities.csv"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv(filepath)
city_data.head()

,City,Temperature_Max,Latitude,Longitude,Humidity %,Cloudiness %,Wind Speed
0,Pakse,33.00,15.12,105.78,46,40,1.00
1,Taltal,15.27,-25.40,-70.48,89,7,2.00
2,Bredasdorp,15.00,-34.53,20.04,93,18,1.00
3,Kapaa,25.00,22.08,-159.32,69,90,4.68
4,Saint Paul Harbor,5.00,57.79,-152.41,48,1,13.90


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = city_data[['Latitude', 'Longitude']].astype(float)
humidity = city_data['Humidity %'].astype(float)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                dissipating=False, max_intensity=100,
                                point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
idealWeather = city_data.loc[city_data['Temperature_Max']>15].loc[city_data['Cloudiness %']<25].loc[city_data['Humidity %']<25].loc[city_data['Wind Speed']<10].reset_index(drop=True)
idealWeather 

,City,Temperature_Max,Latitude,Longitude,Humidity %,Cloudiness %,Wind Speed
0,Muroto-misakicho,19.00,33.28,134.15,19,20,8.20
1,Ojinaga,21.00,29.57,-104.42,21,1,4.60
2,Jiménez,20.87,28.33,-105.40,13,15,3.19
3,Zhangjiakou Shi Xuanhua Qu,16.37,40.61,115.04,12,0,1.59
4,Illela,24.77,13.73,5.30,20,14,0.21
5,Tahoua,28.00,14.89,5.27,23,14,1.00
6,Marabba,27.67,12.35,32.18,9,0,5.61
7,Kayes,23.77,14.00,-11.00,23,12,1.38
8,Taoudenni,16.17,22.68,-3.98,19,11,3.27


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame()
hotel_df['Hotel Name'] = ""

In [7]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "keyword": "hotel",
    "type": "lodging",
    "key": g_key
    }

# Use the lat/lng we recovered to identify hotels
for index, row in idealWeather.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel = requests.get(base_url, params=params)
    
    # convert to json
    hotel = hotel.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = hotel['results'][0]['name']
        hotel_df.loc[index, "Lat"] = hotel['results'][0]['geometry']['location']['lat']
        hotel_df.loc[index, "Lng"] = hotel['results'][0]['geometry']['location']['lng']

       
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [8]:
hotel_df = hotel_df.dropna().reset_index(drop = True)
hotel_df

,Hotel Name,Lat,Lng
0,ウトコ オーベルジュ＆スパ,33.257897,134.182196
1,Hotel Cañon del Peguis,29.551111,-104.422222
2,Norinco Hotel,40.607256,115.080537
3,HOTEL TARKA,14.883738,5.298127


In [9]:
params = {
    "key": g_key
    }

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["latlng"] = f"{lat},{lng}"
    
    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    # make request and print url
    location = requests.get(base_url, params=params)
    
    # convert to json
    location = location.json()
        
    if location['results'][0]['address_components'][0]['types'] == ['locality','political']:
        hotel_df.loc[index, "City"] = location['results'][0]['address_components'][0]['long_name']
    elif location['results'][0]['address_components'][1]['types'] == ['locality','political']:
        hotel_df.loc[index, "City"] = location['results'][0]['address_components'][1]['long_name']
    elif location['results'][0]['address_components'][2]['types'] == ['locality','political']:
        hotel_df.loc[index, "City"] = location['results'][0]['address_components'][2]['long_name']
    elif location['results'][0]['address_components'][3]['types'] == ['locality','political']:
        hotel_df.loc[index, "City"] = location['results'][0]['address_components'][3]['long_name']
    elif location['results'][0]['address_components'][4]['types'] == ['locality','political']:
        hotel_df.loc[index, "City"] = location['results'][0]['address_components'][4]['long_name']
    elif location['results'][0]['address_components'][5]['types'] == ['locality','political']:
        hotel_df.loc[index, "City"] = location['results'][0]['address_components'][5]['long_name']
        
        
    if location['results'][0]['address_components'][0]['types'] == ['country','political']:
         hotel_df.loc[index, "Country"] = location['results'][0]['address_components'][0]['long_name']
    elif location['results'][0]['address_components'][1]['types'] == ['country', 'political']:
         hotel_df.loc[index, "Country"] = location['results'][0]['address_components'][1]['long_name']
    elif location['results'][0]['address_components'][2]['types'] == ['country', 'political']:
        city.loc[index, "Country"] = location['results'][0]['address_components'][2]['long_name']
    elif location['results'][0]['address_components'][3]['types'] == ['country', 'political']:
        hotel_df.loc[index, "Country"] = location['results'][0]['address_components'][3]['long_name']
    elif location['results'][0]['address_components'][4]['types'] == ['country', 'political']:
        hotel_df.loc[index, "Country"] = location['results'][0]['address_components'][4]['long_name']
    elif location['results'][0]['address_components'][5]['types'] == ['country', 'political']:
        hotel_df.loc[index, "Country"] = location['results'][0]['address_components'][5]['long_name']
    elif location['results'][0]['address_components'][6]['types'] == ['country', 'political']:
        hotel_df.loc[index, "Country"] = location['results'][0]['address_components'][6]['long_name']

In [10]:
hotel_df

,Hotel Name,Lat,Lng,City,Country
0,ウトコ オーベルジュ＆スパ,33.257897,134.182196,Muroto,Japan
1,Hotel Cañon del Peguis,29.551111,-104.422222,Ojinaga,Mexico
2,Norinco Hotel,40.607256,115.080537,Zhangjiakou Shi,China
3,HOTEL TARKA,14.883738,5.298127,Tahoua,Niger


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers_layer)
fig

fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers_layer)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))